In [13]:
import time
import board
import csv
import serial
import sys
import adafruit_bme680
from adafruit_pm25.uart import PM25_UART
import RPi.GPIO as GPIO

# Setup for command line arguments

runtime = int(input("How many seconds to run?"))
filename = str(input("File name?"))
interval = int(input('How long is each interval?'))

# Setup BME680 Sensor
i2c = board.I2C()  # Uses board.SCL and board.SDA
bme680 = adafruit_bme680.Adafruit_BME680_I2C(i2c)
bme680.sea_level_pressure = 1013.25

# Setup PM2.5 Sensor
uart = serial.Serial("/dev/ttyS0", baudrate=9600, timeout=0.25)
pm25 = PM25_UART(uart, reset_pin=None)

# Setup GPIO for counting
GPIO.setmode(GPIO.BCM)
try: 
    GPIO.remove_event_detect(gpio_pin)
except RunTimeError:
    pass
    
GPIO.setup(17, GPIO.IN)
GPIO.add_event_detect(17, GPIO.FALLING)

# Prepare CSV file
file = open(filename, 'w', newline='')
writer = csv.writer(file)
meta_data = ['Time', 'PM2.5', 'PM10', 'Temperature', 'Pressure', 'Humidity', 'Altitude', 'GPIO Counts']
writer.writerow(meta_data)

start_time = time.time()
now = start_time
interval = 1  # Data collection interval in seconds
counts = 0  # Initialize GPIO counts

# Collect data until run_time is reached
while (now - start_time) < runtime:
    time.sleep(interval)
    
    # PM2.5 data
    try:
        aqdata = pm25.read()
        data25um = aqdata["particles 25um"]
        data10um = aqdata["particles 10um"]
    except RuntimeError:
        print("Failed to read from PM2.5 sensor.")
        data25um, data10um = (None, None)
    
    # BME680 data
    temp = bme680.temperature
    humidity = bme680.relative_humidity
    press = bme680.pressure
    alt = bme680.altitude

    # GPIO count
    current_counts = counts
    counts = 0  # Reset counts after each interval

    # Timestamp
    now = time.time()
    
    # Write data to CSV
    datalist = [now, data25um, data10um, temp, press, humidity, alt, current_counts]
    writer.writerow(datalist)
    print(datalist)

    # Reset GPIO counts for the next interval
    if GPIO.event_detected(17):
        counts += 1

file.close()
GPIO.cleanup()
print("Data collection complete.")

How many seconds to run? 10
File name? 1
How long is each interval? 2


NameError: name 'RunTimeError' is not defined